# Just an example.You can alter sample code anywhere.

## Mount your google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# You need to modify this part to the directory where your code is located
%cd "/content/drive/MyDrive/DL_Lab1/"

/content/drive/MyDrive/DL_Lab1


## Import packages

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd

In [4]:
#Fix the random seed
np.random.seed(0)

## Load the data and label

In [5]:
train_load = np.loadtxt('./data/kmnist-train.csv',delimiter=',',dtype="int")
train_data=train_load[:,1:]
train_label=train_load[:,0]
print("shape of train_data: {}".format(train_data.shape))
print("shape of train_label: {}".format(train_label.shape))

shape of train_data: (60000, 784)
shape of train_label: (60000,)


## Show the training data

In [6]:
# uncomment if you want to show the training data
#plt.figure(figsize=(20, 20))
#for index in range(10):
#    image = train_data[index+20000].reshape(28,28)
#    plt.subplot(2, 5, index+1)
#    plt.imshow(image)
#plt.show()

In [7]:
train_image_num = train_data.shape[0]
train_data = train_data.astype('float32')

print("train_image_num  is : {}".format(train_image_num))

train_image_num  is : 60000


## Change numpy array to pytorch tensor

In [8]:
train_data_tensor = torch.from_numpy(train_data)
train_label_tensor = torch.from_numpy(train_label)

## Validation image number

In [9]:
val_image_num=10000

## Convert labels to one hot vector


In [10]:
label_temp = np.zeros((train_image_num, 10), dtype = np.float32)
for i in range(train_image_num):
    label_temp[i][train_label[i]] = 1
train_label_onehot = np.copy(label_temp)
train_label_onehot_tensor = torch.from_numpy(train_label_onehot)
print("One-hot training labels shape:",train_label_onehot.shape)


One-hot training labels shape: (60000, 10)


## Hyperparameters

In [11]:
EPOCH = 200
Batch_size = 50 # 10000 should be divisible by batch_size
Learning_rate = 1e-3

## Define the models with pytorch

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1 * 28 * 28, 1024)  # Adjust input size to 1*28*28 for flattened images
        self.fc2 = nn.Linear(1024, 10)  # 10 output classes (for MNIST)

    def forward(self, x):
        # Flatten input from (batch_size, 1, 28, 28) to (batch_size, 784)
        x = x.view(-1, 1 * 28 * 28)

        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # Final layer without activation (softmax applied later if needed)
        return x

# Instantiate the network
net = Net()

## Criterion and Optimizer

In [13]:
import torch.optim as optim


criterion = nn.CrossEntropyLoss()  # Assuming classification task
optimizer = optim.SGD(net.parameters(), lr=Learning_rate)

## Training

In [14]:

train_batch_num = (train_image_num - val_image_num )//Batch_size
val_batch_num = (val_image_num)//Batch_size

for epoch in range(1, EPOCH+1):
    train_hit = 0
    val_hit = 0
    total_train_loss = 0.0
    total_val_loss = 0.0
    for it in range(train_batch_num):
        optimizer.zero_grad()
        outputs = net(train_data_tensor[it*Batch_size:(it+1)*Batch_size])
        _, pred_index = torch.max(outputs, 1)
        train_hit += (pred_index == train_label_tensor[it*Batch_size:(it+1)*Batch_size]).sum().item()
        loss = criterion(outputs, train_label_onehot_tensor[it*Batch_size:(it+1)*Batch_size])
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    with torch.no_grad():
        for titt in range(val_batch_num):
            tit=train_batch_num+titt
            outputs = net(train_data_tensor[tit*Batch_size:(tit+1)*Batch_size])
            _, pred_index = torch.max(outputs, 1)
            val_hit += (pred_index == train_label_tensor[tit*Batch_size:(tit+1)*Batch_size]).sum().item()
            loss = criterion(outputs, train_label_onehot_tensor[tit*Batch_size:(tit+1)*Batch_size])
            total_val_loss += loss.item()

    print('Task2  | Epoch:%3d'%epoch, ' |Train Loss:%8.4f'%(total_train_loss/train_batch_num), ' |Train Acc:%3.4f'%(train_hit/(train_image_num-val_image_num)*100.0)
          , ' |Val Loss:%8.4f'%(total_val_loss/val_batch_num), ' |Val Acc:%3.4f'%(val_hit/val_image_num*100.0))

Task2  | Epoch:  1  |Train Loss:  1.6340  |Train Acc:84.6820  |Val Loss:  0.4221  |Val Acc:89.8200
Task2  | Epoch:  2  |Train Loss:  0.2096  |Train Acc:94.5180  |Val Loss:  0.3484  |Val Acc:91.7600
Task2  | Epoch:  3  |Train Loss:  0.0931  |Train Acc:97.6040  |Val Loss:  0.3321  |Val Acc:92.2800
Task2  | Epoch:  4  |Train Loss:  0.0453  |Train Acc:99.0420  |Val Loss:  0.3297  |Val Acc:92.7200
Task2  | Epoch:  5  |Train Loss:  0.0238  |Train Acc:99.6360  |Val Loss:  0.3320  |Val Acc:92.8000
Task2  | Epoch:  6  |Train Loss:  0.0133  |Train Acc:99.8800  |Val Loss:  0.3323  |Val Acc:93.0600
Task2  | Epoch:  7  |Train Loss:  0.0085  |Train Acc:99.9600  |Val Loss:  0.3326  |Val Acc:93.3100
Task2  | Epoch:  8  |Train Loss:  0.0061  |Train Acc:99.9760  |Val Loss:  0.3351  |Val Acc:93.4000
Task2  | Epoch:  9  |Train Loss:  0.0047  |Train Acc:99.9860  |Val Loss:  0.3371  |Val Acc:93.4500
Task2  | Epoch: 10  |Train Loss:  0.0039  |Train Acc:99.9920  |Val Loss:  0.3389  |Val Acc:93.4900
Task2  | E